In [37]:

import functools
import traceback


def log_exceptions(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        try:
            result = func(*args, **kwargs)
        except Exception as e:
            print(f"FAILURE: {func_name} failed: {e}")
            traceback.print_exc()
        else:
            print(f"SUCCESS: {func_name} succeeded")
            return result
    return wrapper


def decorate_methods(decorator):
    def decorate(obj):
        if isinstance(obj, type):
            # Decorate class methods
            for name, method in vars(obj).items():
                if callable(method):
                    setattr(obj, name, decorator(method))
            return obj
        else:
            # Decorate standalone functions
            return decorator(obj)
    return decorate


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os 
import sys 

# path to this file's directory and parent directory
current_directory = os.getcwd()
base_directory = os.path.dirname(current_directory)

# parent directory of base_directory
parent_directory = os.path.dirname(base_directory)

# add to path
sys.path.append(base_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(base_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(base_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(base_directory, 'quick_batch', 'queue_app')


In [3]:
parent_directory

'/Users/wattjer/Desktop'

In [46]:
import shutil
import os
import sys
import docker 


# path to this file's directory and parent directory
current_directory = os.getcwd()
base_directory = os.path.dirname(current_directory)

# add to path
sys.path.append(base_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(base_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(base_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(base_directory, 'quick_batch', 'queue_app')


# create client for docker
@log_exceptions
def create_client():
    return docker.from_env()

# remove all docker containers
@log_exceptions
def remove_all_containers(client):
    for container in client.containers.list(all=True):
        container.remove(force=True)

# build processor_app docker image
@log_exceptions
def build_processor_image(client, processor_path):
    # copy processor requirements.txt to processor_app directory to roll into dockerfile
    shutil.copyfile(os.path.join(base_directory, 'processor_requirements.txt'), os.path.join(processor_path, 'processor_requirements.txt'));

    # create docker image for processor app - including user defined requirements
    client.images.build(path=processor_path, tag='quick_batch_processor_app', quiet=False)

    # remove processor_requirements.txt from the processor_path directory 
    os.remove(os.path.join(processor_path, 'processor_requirements.txt'))
    
@log_exceptions
def build_queue_image(client, queue_path):
    # create docker image for queue app - including user defined requirements
    client.images.build(path=queue_path, tag='quick_batch_queue_app', quiet=False)
    
@log_exceptions
def build_controller_image(client, controller_path):
    # create docker image for queue app - including user defined requirements
    client.images.build(path=controller_path, tag='quick_batch_controller_app', quiet=False)

In [74]:
client = create_client()

build_processor_image(client, 
                      processor_path)

build_queue_image(client,
                  queue_path)

build_controller_image(client,
                       controller_path)


# remove_all_containers(client)

SUCCESS: create_client succeeded
SUCCESS: build_processor_image succeeded
SUCCESS: build_queue_image succeeded
SUCCESS: build_controller_image succeeded


In [56]:
# start processor_app docker container with interactive terminal
processor_app_test = client.containers.run('quick_batch_processor_app', 
                                           detach=True, 
                                           name='processor_app', 
                                           tty=True, 
                                           stdin_open=True, 
                                           ports={'5000/tcp': 5000},
                                           volumes={
                                               processor_path + '/processor_app': 
                                               {'bind': '/my_app', 'mode': 'rw'},
                                               base_directory + '/processor.py':
                                               {'bind': '/my_app/processor.py', 'mode': 'rw'},
                                           },
                                           command = 'python /my_app/run.py'
                                          );

In [103]:
# start queue_app docker container with interactive terminal
queue_app_test = client.containers.run('quick_batch_queue_app',
                                        detach=True,
                                        name='queue_app',
                                        tty=True,
                                        stdin_open=True,
                                        ports={'80/tcp': 80},
                                        volumes={
                                            queue_path + '/queue_app':  
                                            {'bind': '/my_app', 'mode': 'rw'},
                                            base_directory + '/processor.py':
                                            {'bind': '/my_app/processor.py', 'mode': 'rw'},             
                                        },
                                        command = 'python /my_app/run.py'
                                        );

In [93]:
queue_app_test.exec_run(cmd='cat /my_app/queue_app/processor.py',
                        detach=True,
                        stream=True,
                        tty=True,
                        stdout=True,
                        stderr=True)

ExecResult(exit_code=None, output='')

In [85]:
print(queue_app_test.logs())

b'Python 3.10.11 (main, May  3 2023, 08:40:14) [GCC 10.2.1 20210110] on linux\r\nType "help", "copyright", "credits" or "license" for more information.\r\n'


In [102]:
remove_all_containers(client)

SUCCESS: remove_all_containers succeeded


In [36]:
# start process
processor_app_test

'processor_app'

In [ ]:
# execute hello_world.py in processor_app docker container
processor_app_test.exec_run('python3 -m processor_app.hello_world')

In [32]:
# list running docker containers by name
client.containers.list(all=True, filters={'name': 'processor_app'},)


TypeError: list() got an unexpected keyword argument 'status'

In [28]:
# list current docker images 
client.images.list()

In [ ]:
a